In [1]:
# ! pip install plotly==5.22.0
# ! pip install jupyterlab-widgets==1.1.1 ipywidgets==7.7.2
# #! pip install jupyterlab-widgets==3.0.11 ipywidgets==8.1.3
# ! pip install umap-learn
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#! jupyter labextension list
#! jupyter nbextension list

### In this example, the embeddings are generated from a Barlow Twin model trained on the Left CINGULATE region on UKB for the UKB and HCP datasets.

 -For the HCP dataset, the associated buckets can be found in: 

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Lbuckets

 -For the UKB dataset, the buckets are not generated yet, but it would be found in:

/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lbuckets

### Load your data

In [3]:
#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/hcp_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/UKB_epoch60_embeddings/full_embeddings.csv", index_col=0)

#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/08_Heritability_Left_Orbital_HCP_dim256/Output/20-56-02_1/HCP_random_epoch70_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/08_Heritability_Left_Orbital_HCP_dim256/Output/20-56-02_1/UKB_random_epoch70_embeddings/full_embeddings.csv", index_col=0)
#embeddings_HCP = pd.read_csv("/neurospin/dico/data/deep_folding/current/models/Champollion_V0_drop_rate_0/FIP_right/18-20-41_215/FIP_custom_embeddings/custom_cross_val_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/data/deep_folding/current/models/Champollion_V0_drop_rate_0/FIP_right/18-20-41_215/ukb_random_embeddings/full_embeddings.csv", index_col=0)
embeddings_HCP = pd.read_csv("/volatile/jl277509/Runs/02_STS_babies/Program/Output/PCS_FINAL_MODEL/19-12-25_3/CCD_random_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB = pd.read_csv("/volatile/jl277509/Runs/02_STS_babies/Program/Output/PCS_FINAL_MODEL/19-12-25_3/UKB_random_embeddings/full_embeddings.csv", index_col=0)

embeddings_UKB.head()

embeddings_HCP = embeddings_HCP
embeddings_UKB = embeddings_UKB
embeddings_UKB.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
sub-1000021,15.592494,34.494900,8.719974,84.042755,-2.338658,17.109613,17.615273,-1.569066,-0.724397,-3.401615,...,21.493185,-21.094856,-16.412336,0.503177,15.058713,-6.507754,-56.560260,-0.500706,22.970040,-0.897924
sub-1000458,21.859705,41.393238,16.137548,58.983418,14.475472,34.597996,-0.670761,-17.257439,2.672007,-8.064828,...,33.590267,-7.286129,-16.044346,-6.735470,13.279847,23.086065,-18.885305,15.098557,15.267213,-12.603914
sub-1000575,66.839590,-7.664413,16.941942,52.955128,-38.902386,-7.481332,-6.376166,2.011489,2.837864,8.470457,...,-26.563917,4.246506,-2.606314,-13.200418,0.386144,-7.431461,7.049025,20.465460,15.058507,21.503077
sub-1000606,41.851505,10.870255,31.636250,55.456170,13.537527,4.086966,33.851727,-4.856818,-23.671595,-4.551562,...,-15.102870,-13.294502,-0.053844,6.308680,26.031740,-13.061463,-39.270817,3.099191,3.324039,5.225338
sub-1000963,16.002398,13.636017,15.947115,71.199370,1.879433,28.217922,1.475944,6.490414,7.099989,1.988958,...,7.824328,-4.322985,-28.375147,31.262850,11.012692,-0.501506,-29.956993,21.468794,20.046604,-0.328055


### Scale your data

In [4]:
scaler = StandardScaler()
scaler.fit(embeddings_UKB)

scl_bdd_hcp = scaler.transform(embeddings_HCP)
scl_bdd_ukb = scaler.transform(embeddings_UKB)

### 2D UMAP

In [5]:
reducer2D = umap.UMAP(n_components=2)

reducer2D.fit(scl_bdd_ukb)

bdd_2D_HCP = reducer2D.transform(scl_bdd_hcp)
bdd_2D_UKB = reducer2D.transform(scl_bdd_ukb)

/casa/home/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [7]:
bdd_2D_HCP = pd.DataFrame(bdd_2D_HCP, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])

bdd_2D_HCP['Dataset'] = 'CCD'
bdd_2D_UKB['Dataset'] = 'UkBioBank'

bdd_2D_HCP['ID'] = embeddings_HCP.index
bdd_2D_UKB['ID'] = embeddings_UKB.index

bdd_2D_All = pd.concat([bdd_2D_UKB,bdd_2D_HCP], axis=0)
bdd_2D_All.head()

,Dim 1,Dim 2,Dataset,ID
0,6.522198,2.673499,UkBioBank,sub-1000021
1,1.457132,4.815637,UkBioBank,sub-1000458
2,3.405086,6.606355,UkBioBank,sub-1000575
3,5.150643,4.625624,UkBioBank,sub-1000606
4,-0.205544,3.092979,UkBioBank,sub-1000963


### Plot it in the notebook or write a html file

In [8]:
subject_id_list = []
dataset_name_list = []

In [9]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_All, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='2D UMAP HCP and UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

#fig.show()

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ['UkBioBank', 'sub-1000575'],
                                   ...,
                                   ['UkBioBank', 'sub-6023847'],
                                   ['UkBioBank', 'sub-6024038'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'a6dea1b3-1123-4a18-8525-756f9af7fae5',
              'x': array([6.5221977, 1.4571319, 3.4050858, ..., 4.535739 , 1.4252585, 1.233976 ],
               

In [9]:
print(subject_id_list, dataset_name_list)

[760551, 246133, 833148] ['hcp', 'hcp', 'hcp']


In [10]:
#subject_id_list = [214423, 849971, 208630, 683256, 589567]
#dataset_name_list = ['hcp' for i in range(5)]

In [11]:
side = "R"
region = "F.I.P." #"CINGULATE"
nb_columns = 4


import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims

def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()



bucket_files = []

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'

    if dataset.lower() in ['hcp']:
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset.lower()}/crops/2mm/{region}/mask/{side}crops'

    filename = f'{path}/{subject_id}_cropped_skeleton.nii.gz'#.minf'

    if os.path. isfile(filename):
        bucket_files.append(filename)
    else:
        print(f"{filename} is not a correct path, or the .bck doesn't exist")
# print(bucket_files)

#####################################
# We create the blocks
#####################################

block = a.createWindowsBlock(nb_columns) # nb of columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = to_bucket(a.loadObject(file))
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])


#####################################
# We compute here the average
#####################################

dic_vol = {}
dim = 0
rep = 0
while dim == 0 and rep < len(subject_id_list):
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_name_list[rep]}/crops/2mm/{region}/mask/{side}crops"
    if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
        sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
        dim = sum_vol.shape
        sum_vol.fill(0)
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
    rep += 1

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
    elif dataset.lower() == 'hcp':
        dataset = 'hcp'
        
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

    if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_# We creat the blockscropped_skeleton.nii.gz'):
        vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        # compare the dim with the first file dim

        if vol.np.shape != dim:
            raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

            
        # to have a binary 3D structure
        dic_vol[subject_id] = (vol.np > 0).astype(int)
        sum_vol.np[:] += (vol.np > 0).astype(int) 
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

sum_vol = sum_vol / len(subject_id_list)
print(sum_vol, sum_vol.shape)


a_sum_vol = a.toAObject(sum_vol)
a_sum_vol.setPalette(minVal=0, absoluteMode=True)
wsum = a.createWindow('Sagittal', block=block)
wsum.addObjects(a_sum_vol)
rvol = a.fusionObjects(objects=[a_sum_vol], method='VolumeRenderingFusionMethod')
rvol.releaseAppRef()
# custom palette
n = len(subject_id_list)
pal = a.createPalette('VR-palette')
pal.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
build_gradient(pal)
rvol.setPalette('VR-palette', minVal=0, absoluteMode=True)
pal2 = a.createPalette('slice-palette')
pal2.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
build_gradient(pal2)
a_sum_vol.setPalette('slice-palette')
# rvol.palette().fill()
wvr = a.createWindow('3D', block=block)
wvr.addObjects(rvol)
#print(dic_vol[subject_id_list[0]].shape)
#print(np.count_nonzero(dic_vol[subject_id_list[0]]))

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols
loading module statsplotwindow
loading module foldsplit
loading module histogram
loading module save_resampled
loading module simple_controls
loading module valuesplotwindow
loading module gradientpalette
all python modules loaded
Anatomist started.
FileNotFound /neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rcrops/760551_cropped_skeleton.nii.gz
FileNotFound /neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rcrops/246133_cropped_skeleton.nii.gz
FileNotFound /neurospin/dico/data/deep_folding/current/datasets/hcp/c

: 

In [13]:
# Create axis
axes = list(dim[:3])
dim1 = list(dim[:3])[0]
dim2 = list(dim[:3])[1]
dim3 = list(dim[:3])[2]

# Create Data
data = sum_vol.np.reshape(list(dim[:3]))

X, Y, Z = np.mgrid[0:dim1:1, 0:dim2:1, 0:dim3:1]
values = np.flip(data, axis=[1,2])

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.1,
    isomax=1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()

TypeError: 'int' object is not subscriptable

In [ ]:
"""
data_filenames_str = ' '.join(data_filenames_list)

#os.system(f'anatomist --input {data_filenames_str}')

# Command to run the Python script inside the container
command = f'bv bash -c "python visu_anatomist.py {data_filenames_str}"'

# Execute the command
os.system(command)
"""

In [ ]:
# See for more information
# https://plotly.com/python/line-and-scatter/
# https://plotly.com/python/setting-graph-size/
# app.run_server(debug=True)